# Sampling and Splitting

## Sampling

Sampling for basic tabular datasets. (Not designed for time series as of now.)

In [1]:
import polars as pl
import polars_ds as pds
import polars_ds.sample_and_split as ss

In [2]:
df = pds.frame(size=100_000).with_columns(
    pds.random(0.0, 12.0).alias("uniform_1"),
    pds.random(0.0, 1.0).alias("uniform_2"),
    pds.random_exp(0.5).alias("exp"),
    pds.random_normal(0.0, 1.0).alias("normal"),
    pds.random_normal(0.0, 1000.0).alias("fat_normal"),
    (pds.random_int(0, 3)).alias("flags"),
    pl.Series(["A"] * 30_000 + ["B"] * 30_000 + ["C"] * 40_000).alias("category"),
)
df.head()

row_num,uniform_1,uniform_2,exp,normal,fat_normal,flags,category
i64,f64,f64,f64,f64,f64,i32,str
0,6.123157,0.01365,0.485073,0.524003,-703.178104,0,"""A"""
1,2.775694,0.690248,2.850941,-1.966617,-143.788506,0,"""A"""
2,2.049679,0.557566,2.132329,-0.160845,-71.074823,1,"""A"""
3,11.261876,0.561843,4.737346,1.611832,-300.312419,2,"""A"""
4,7.955185,0.223838,1.977746,0.272536,-398.71787,1,"""A"""


In [3]:
ss.random_cols(df.columns, 2, keep = ["row_num"])

['row_num', 'uniform_2', 'exp']

In [4]:
# Random Sample
ss.sample(df, 0.6) # by ratio

row_num,uniform_1,uniform_2,exp,normal,fat_normal,flags,category
i64,f64,f64,f64,f64,f64,i32,str
1,2.775694,0.690248,2.850941,-1.966617,-143.788506,0,"""A"""
2,2.049679,0.557566,2.132329,-0.160845,-71.074823,1,"""A"""
3,11.261876,0.561843,4.737346,1.611832,-300.312419,2,"""A"""
4,7.955185,0.223838,1.977746,0.272536,-398.71787,1,"""A"""
6,3.576043,0.148358,0.240111,-1.077243,-1401.035922,0,"""A"""
…,…,…,…,…,…,…,…
99990,1.210644,0.245434,2.107333,-0.943427,-105.682645,0,"""C"""
99994,7.409102,0.831871,1.337031,0.203354,-616.625619,0,"""C"""
99995,2.374191,0.994781,0.468766,-1.096312,579.203078,0,"""C"""


In [5]:
ss.sample(df, 30_000) # by count

row_num,uniform_1,uniform_2,exp,normal,fat_normal,flags,category
i64,f64,f64,f64,f64,f64,i32,str
1,2.775694,0.690248,2.850941,-1.966617,-143.788506,0,"""A"""
2,2.049679,0.557566,2.132329,-0.160845,-71.074823,1,"""A"""
5,1.6969,0.033205,1.538025,1.208673,-445.691216,0,"""A"""
7,8.997335,0.862542,0.442738,1.143512,1912.542472,1,"""A"""
9,6.104426,0.953088,4.059342,-1.483365,-1665.720052,0,"""A"""
…,…,…,…,…,…,…,…
99990,1.210644,0.245434,2.107333,-0.943427,-105.682645,0,"""C"""
99995,2.374191,0.994781,0.468766,-1.096312,579.203078,0,"""C"""
99997,4.247473,0.057653,0.554583,1.383503,-925.246072,0,"""C"""


In [6]:
df.group_by("flags").len().sort("flags")

flags,len
i32,u32
0,33569
1,33084
2,33347


In [7]:
# Downsample on one group
sa1 = ss.downsample(
    df, 
    (pl.col("flags") == 0, 0.5)
)
sa1.group_by("flags").len().sort("flags")

flags,len
i32,u32
0,16784
1,33084
2,33347


In [8]:
# Downsample on multiple groups
sa2 = ss.downsample(
    df, 
    (pl.col("flags") == 0, 0.5),
    (pl.col("flags") == 1, 0.3),
    (pl.col("flags") == 2, 0.4),
)
sa2.group_by("flags").len().sort("flags")

flags,len
i32,u32
0,16784
1,9925
2,13338


In [9]:
df.group_by("category").len().sort("category")

category,len
str,u32
"""A""",30000
"""B""",30000
"""C""",40000


In [10]:
# Volume neutral by each category, will take the greatest possible value so that we get neutral volume.
vn = ss.volume_neutral(
    df,
    by = pl.col("category"),
)
vn.group_by("category").len().sort("category")

category,len
str,u32
"""A""",30000
"""B""",30000
"""C""",30000


In [11]:
# Volume neutral (10_000) by each category
vn = ss.volume_neutral(
    df,
    by = pl.col("category"),
    target_volume = 10_000
)
vn.group_by("category").len().sort("category")

category,len
str,u32
"""A""",10000
"""B""",10000
"""C""",10000


In [12]:
# Volume neutral (10_000) by a more complicated condition
vn = ss.volume_neutral(
    df,
    by = pl.col("category") == "A",
    target_volume = 10_000
) # This makes sense because count for B + count for C = 10_000
vn.group_by("category").len().sort("category")

category,len
str,u32
"""A""",10000
"""B""",4219
"""C""",5781


In [13]:
# Volume neutral sample with a control level. Volume neutral happens under the category level, meaning
# the volume for each flag in each category is neutral.
vn = ss.volume_neutral(
    df,
    by = pl.col("flags"),
    control = pl.col("category")
) 
vn.group_by(["category", "flags"]).len().sort(["category", "flags"])

category,flags,len
str,i32,u32
"""A""",0,9954
"""A""",1,9954
"""A""",2,9954
"""B""",0,9910
"""B""",1,9910
"""B""",2,9910
"""C""",0,13215
"""C""",1,13215
"""C""",2,13215


In [14]:
# We may not meet the target volume for all categories.
vn = ss.volume_neutral(
    df,
    by = pl.col("flags"),
    control = pl.col("category"),
    target_volume= 10_000
) # 
vn.group_by(["category", "flags"]).len().sort(["category", "flags"])

category,flags,len
str,i32,u32
"""A""",0,9954
"""A""",1,9954
"""A""",2,9954
"""B""",0,9910
"""B""",1,9910
"""B""",2,9910
"""C""",0,10000
"""C""",1,10000
"""C""",2,10000


## Splitting

Split by ratios.

In [15]:
print(df.shape)
train, test = ss.split_by_ratio(
    df,
    split_ratio = 0.6
)
print(train.shape)
print(test.shape)


(100000, 8)
(60000, 8)
(40000, 8)


In [16]:
print(df.shape)
for frame in ss.split_by_ratio(df, split_ratio = [0.25, 0.4, 0.10, 0.25]):
    print(frame.shape)

(100000, 8)
(25000, 8)
(40000, 8)
(10000, 8)
(25000, 8)
